In [1]:
import configparser
import sys
from pathlib import Path
from typing import Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlalchemy
import sqlite3

# import sqlite_utils

%load_ext sql

In [2]:
sys.path.append(str(Path.cwd().parent / 'src'))

from db_declaration import Base, Record, Artist

<class 'sqlalchemy.ext.declarative.api.DeclarativeMeta'>


In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.style.use('raph-base')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

np.random.seed(666)

In [4]:
print(sys.executable)
print(sys.version)
print(f'sqlalchemy {sqlalchemy.__version__}')

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]
sqlalchemy 1.3.17


## Connect to DB

In [5]:
# Connect to DiscoBase.db the sqlite way

db_path = "../db_aka_discobase/DiscoBase.db"
sqlite_conn_str = f"sqlite:///{db_path}"

# Connect to sqlalchemy
engine = sqlalchemy.create_engine(sqlite_conn_str)

In [23]:
# Connect to ipython-sql
%sql $sqlite_conn_str

'Connected: @../db_aka_discobase/DiscoBase.db'

In [7]:
query = ("""
SELECT 
  m.name AS table_name, 
  p.name AS column_name,
  p.type AS data_type,
  p.'notnull' AS null_constraint,
  p.dflt_value AS default_value,
  p.pk AS primary_key
FROM 
  sqlite_master AS m
JOIN 
  pragma_table_info(m.name) AS p
ORDER BY 
  m.name, 
  p.cid
;"""
)

In [8]:
%sql $query

 * sqlite:///../db_aka_discobase/DiscoBase.db
Done.


table_name,column_name,data_type,null_constraint,default_value,primary_key
artist_genre_link,artist_id,INTEGER,1,None,1
artist_genre_link,genre_id,INTEGER,1,None,2
artist_label_link,artist_id,INTEGER,1,None,1
artist_label_link,label_id,INTEGER,1,None,2
artists,artist_id,INTEGER,1,None,1
artists,artist_name,TEXT,1,None,0
artists,artist_country,TEXT,0,None,0
formats,format_id,INTEGER,1,None,1
formats,format_name,TEXT,1,None,0
genre_label_link,genre_id,INTEGER,1,None,1


In [41]:
%sql SELECT COUNT(*) FROM records

 * sqlite:///../db_aka_discobase/DiscoBase.db
Done.


COUNT(*)
293


In [42]:
%sql SELECT * FROM records WHERE title = 'Death Lust'

 * sqlite:///../db_aka_discobase/DiscoBase.db
Done.


record_id,artist_id,title,genre_id,label,year,format_id,vinyl_color,lim_edition,number,remarks,purchase_date,price,digitized,rating,active
264,167,Death Lust,1,None,2020,7,"milky clear with blood red, black and white splatter",250,nan,"warped, etched on B-Side",2020-07-01,20.0,1,8,1


In [43]:
%sql SELECT * FROM artists ORDER BY artist_name

 * sqlite:///../db_aka_discobase/DiscoBase.db
Done.


artist_id,artist_name,artist_country
139,Abuse of Power,None
91,Advent,None
68,Agnosy,None
59,All For Nothing,None
63,Ancient,None
77,Arms Race,None
24,Asphyx,None
25,At The Gates,None
48,Ays,None
86,Baboon Show,None


In [44]:
%sql SELECT * FROM genres ORDER BY genre_name

 * sqlite:///../db_aka_discobase/DiscoBase.db
Done.


genre_id,genre_name
3,Black Metal
8,Crossover
7,Crust
1,Death Metal
9,Grindcore
5,Hardcore
6,Punk
4,Speed Metal
2,Thrash Metal
